In [143]:
import pandas as pd
import pyterrier as pt
import yaml
import os
from src.load_index import load_index, load_topics, load_qrels, tag
from src.extend_runs import extend_run_full
import sqlite3
from repro_eval.Evaluator import RpdEvaluator
import pytrec_eval
import numpy as np

from repro_eval.util import arp, arp_scores

if not pt.started():
    pt.init()

In [2]:
with open("data/LongEval/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

results_path = "data/results/filter_not_rel/"

index = load_index("t3")
topics = load_topics("t3", "test")

>>> Loaded index with  2049729 documents.


### Create baseline with 1500 docs per query

In [4]:
# Extended baseine for filtering
# BM25 = pt.BatchRetrieve(index, wmodel="BM25", verbose=True, num_results=1500)

# pt.io.write_results(
#     BM25(topics), results_path + f"/CIR_BM25_D-t3_T-t3-long"
# )

### Extend with docids and qids

In [5]:
# Extend the run
# extend_run_full(results_path + f"CIR_BM25_D-t3_T-t3-long")

### Extend with qrels

In [92]:
run = pd.read_csv(results_path + f"CIR_BM25_D-t3_T-t3-l_extended.t3", sep=" ")

/var/folders/pz/f66p4s0s11z2c1c1l3v0qbh40000gn/T/ipykernel_1335/3447229843.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  run = pd.read_csv(results_path + f"CIR_BM25_D-t3_T-t3-l_extended.t3", sep=" ")


In [93]:
conn = sqlite3.connect("data/database.db")
query = "SELECT * FROM qrel"
qrels = pd.read_sql_query(query, conn)

In [94]:
qrels["key"] = qrels["queryid"]+qrels["docid"]

In [95]:
qmap=qrels[["key", "relevance"]].set_index("key").to_dict()["relevance"]

In [96]:
def get_qrel(row, subcollection):
    query_id = row[f"queryid_{subcollection}"]
    doc_id = row[f"docid_{subcollection}"]
    if isinstance(query_id, str) and isinstance(doc_id, str):
        return qmap.get(row[f"queryid_{subcollection}"] + row[f"docid_{subcollection}"], None)
    else:
        return None
       
for subcollection in ["t0", "t1", "t2", "t3"]:
    run[f"qrel_{subcollection}"] = run.apply(get_qrel, subcollection=subcollection, axis=1)


This adds between 450 to 600 qrels per sub-collection

# Filter not rel

In [97]:
# for subcollection in ["t0", "t1", "t2"]:
history = ["t2", "t1", "t0"]
reranked = run.copy()

for subcollection in history:
    reranked = reranked[reranked[f"qrel_{subcollection}"] != 0.0]

In [98]:
reranked = reranked.sort_values(['queryid','score'], ascending=False).groupby('queryid').head(1000)
reranked['rank'] = reranked.groupby('queryid')['score'].rank(ascending=False).astype(int)


In [99]:
run_name = f"CIR_BM25_D-t3_T-t3-filter-notrel-{"".join(history)}"
run_reranked_path = os.path.join(results_path, run_name)

In [100]:
pt.io.write_results(reranked[["queryid", "0", "docid", "score", "rank", "run"]].rename(columns={"queryid": "qid", "docid": "docno"}), run_reranked_path, format='trec', run_name=run_name)

In [101]:
base_path = "data"

with open(os.path.join(base_path, config["subcollections"]["t3"]["qrels"]["test"]), "r") as f_qrels:
    qrels = pytrec_eval.parse_qrel(f_qrels)
evaluator = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

In [102]:
with open(run_reranked_path) as run_reranked:
    r = pytrec_eval.parse_run(run_reranked)
    scores = evaluator.evaluate(r)
    print( "|", 
          ", ".join(history), "|",
        str(round(arp_scores(scores)["P_10"], 4)), "|",
        str(round(arp_scores(scores)["bpref"], 4)), "|",
        str(round(arp_scores(scores)["ndcg"], 4)), "|"
        )

| t3 | 0.1798 | 0.7784 | 0.3851 |


# Boostrel

In [160]:
def direct_boost(run, history, _lambda=1):
    reranking = run.copy()
    
    # min max normalization per topic
    reranking['score'] = reranking.groupby('queryid')['score'].transform(lambda x : x / x.max())
    
    for subcollection in history:
        # All Relevant
        reranking.loc[reranking[f"qrel_{subcollection}"] > 0, 'score'] = reranking.loc[reranking[f"qrel_{subcollection}"] > 0, 'score'] * _lambda ** 2
        
    reranking = reranking.sort_values(['queryid','score'], ascending=False).groupby('queryid').head(1000)
    reranking['rank'] = reranking.groupby('queryid')['score'].rank(ascending=False).astype(int)
    
    return reranking

In [207]:
history = ["t2"]
_lambda = 1 + 2
reranked = direct_boost(run, history, _lambda=_lambda)

In [208]:
run_name = f"CIR_BM25_D-t3_T-t3-rr-rel-{"".join(history)}"
run_reranked_path = os.path.join(results_path, run_name)

In [209]:
pt.io.write_results(reranked[["queryid", "0", "docid", "score", "rank", "run"]].rename(columns={"queryid": "qid", "docid": "docno"}), run_reranked_path, format='trec', run_name=run_name)

base_path = "data"

with open(os.path.join(base_path, config["subcollections"]["t3"]["qrels"]["test"]), "r") as f_qrels:
    qrels = pytrec_eval.parse_qrel(f_qrels)
evaluator = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

In [210]:
with open(run_reranked_path) as run_reranked:
    r = pytrec_eval.parse_run(run_reranked)
    scores = evaluator.evaluate(r)
    print( "|", 
          str(_lambda), "|",
          ", ".join(history), "|",
        str(round(arp_scores(scores)["P_10"], 4)), "|",
        str(round(arp_scores(scores)["bpref"], 4)), "|",
        str(round(arp_scores(scores)["ndcg"], 4)), "|"
        )

| 3 | t2 | 0.1788 | 0.4493 | 0.3822 |
